In [4]:
import os
import random
from tqdm.notebook import tqdm
from stox.alpha_vantage import AlphaVantageStocks
from stox.ibkr import load_symbol_table, save_symbol_table
import omnifig as fig
fig.initialize()

In [5]:
cfg = fig.create_config('container', 'euro', portfolio_name='fe-aug23.txt')
# cfg = fig.create_config('container', 'euro')
print(cfg)

container:
  _type: context
  gadgets:
    common: {_type: common-stats}
    yahoo:
      _type: scope
      gadgets:
        info: {_type: yahoo-info}
        loader: {_type: yahoo-loader}
        portfolio-loader: <>portfolio
features: {business_summary: Company Description, change_52w: 52 Week Change, company_name: Company
    Name, country: Country, forward_eps: Forward EPS, forward_pe: Forward P/E, held_percent_insiders: Percent
    of Shares Held by Insiders, held_percent_institutions: Percent of Shares Held
    by Institutions, industry: Industry, log_market_cap: Log Market Cap, overall_risk: Overall
    Risk (1-10), peg_ratio: PEG Ratio, perf52w: '52 Week Relative Price (1=52 week
    high, 0=52 week low)', price: Price, recommendation_key: Recommendation Key, recommendation_mean: 'Recommendation
    Mean (1=Strong Buy, 5=Strong Sell)', sector: Sector, shares: Current Shares, target_high_price: Target
    High Price, target_low_price: Target Low Price, target_mean_price: Target

In [6]:
ctx = cfg.peek_create('container')
ctx

| CREATING container type='context'


KeyboardInterrupt: 

In [2]:
# av = AlphaVantageStocks()
# info = av.overview('BBVA')
# info.keys()

In [3]:
symbols_table = load_symbol_table()
eur = {yfsym: info for yfsym, info in symbols_table.items() if info['ibkr-contract']['currency'] == 'EUR'}
len(symbols_table), len(eur)

(454, 239)

In [4]:
adr_srcs = {yfsym: info['adr'] for yfsym, info in symbols_table.items() if 'adr' in info}
adrs = {yfsym: info['adr-src'] for yfsym, info in symbols_table.items() if 'adr-src' in info}
assert all(adr_srcs[adrs[yfsym]] == yfsym for yfsym in adrs)
len(adr_srcs), len(adrs)

(182, 182)

In [5]:
nys = [adr for adr in adrs if symbols_table[adr]['ibkr-contract']['primaryExchange'] in {'NYSE', 'NASDAQ'}]
len(nys)

31

In [7]:
# dl = fig.create_config(_type='downloader/alpha-vantage',).create_silent()
# dl

In [8]:
# dl.download_reports('BBVA', date='230909', pbar=tqdm)

In [6]:
date = '230909'
_av_container_source = fig.create_config(**{'_type': 'context', 'gadgets':  {'loader._type': 'loader/alpha-vantage', 'stats._type': 'stats/alpha-vantage'}})
def create_av_container(yfsym):
	with _av_container_source.silence():
		ctx = _av_container_source.create()
	ctx['ticker'] = yfsym
	ctx['date'] = date
	return ctx
_yf_container_source = fig.create_config(**{'_type': 'context', 'gadgets': {'loader._type': 'yahoo-loader', 'info._type': 'yahoo-info'}})
def create_yf_container(yfsym):
	with _yf_container_source.silence():
		ctx = _yf_container_source.create()
	ctx['ticker'] = yfsym
	ctx['date'] = date
	return ctx

In [7]:
apop = {yfsym: create_av_container(yfsym) for yfsym in nys}
len(apop)

31

In [8]:
# ctx = pop[0]
# ctx = random.choice(pop)
ctx = apop['BBVA']
ctx

Context(ticker, date, {overview}, {income_statement}, {balance_sheet}, {cash_flow}, {earnings}, {earnings_calendar}, {ipo_calendar}, {listing_status}, {sector}, {industry}, {market_cap}, {peg_ratio}, {pe_ratio}, {dividend_yield}, {eps}, {beta}, {profit_margin}, {target_price}, {trailing_pe}, {forward_pe}, {high_52w}, {low_52w}, {av_50d}, {av_200d}, {ebitda}, {description}, {name})

In [10]:
spop = {adr: create_yf_container(adrs[adr]) for adr in nys}
len(spop)

31

In [56]:
tk = random.choice(nys)
s = spop[tk]
a = apop[tk]
tk

'SNY'

In [57]:
s['peg_ratio'], a['peg_ratio']

(1.58, 1.65)

In [58]:
s['forward_pe'], a['forward_pe']

(11.376417, 10.49)

In [59]:
s['beta'], a['beta']

(0.400525, 0.401)

In [60]:
s['company_name'], a['name']

('Sanofi', 'Sanofi ADR')

In [61]:
s['market_cap'], a['market_cap']

(126B EUR, 137B USD)